# Class Parser Workflow Test

This notebook reproduces the workflow from poc.ipynb step by step using the new src/ structure.
Test each component individually and then run the complete pipeline.

## Workflow Overview
1. **Data Extraction** - Extract slides and plan from PDF (deterministic)
2. **Branch A** - With plan: outline_two_pass → mapping_two_pass → enrich → writer
3. **Branch B** - No plan: outline_one_shot → enrich → writer
4. **Export** - Generate DOCX document


In [1]:
# Setup and imports
import os
import json
import logging
import sys
from pathlib import Path

# Configure logging for the src package
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)

# Set our package to DEBUG level for detailed progress
logging.getLogger("src").setLevel(logging.DEBUG)

# Load environment variables
import dotenv
dotenv.load_dotenv()

# Import our src modules
from src import (
    Course, CoursePipeline, Content, ContentSection, 
    MappingItem, SectionSlideMapping, Slides
)
from src.data_extraction import SlidesExtractor, PlanExtractor
from src.llm import (
    OutlineOneShot, OutlineTwoPass, MappingTwoPass, Writer, PromptManager
)
# Ensure src is importable when running the notebook directly
import sys
sys.path.append(str(Path('.').resolve()))

print("✅ All imports successful!")
print(f"📁 Working directory: {os.getcwd()}")

# Check if volume directory exists
volume_path = Path("./volume")
slides_path = volume_path / "slides"
print(f"📂 Volume directory exists: {volume_path.exists()}")
print(f"📂 Slides directory exists: {slides_path.exists()}")

if slides_path.exists():
    pdf_files = list(slides_path.glob("*.pdf"))
    print(f"📄 PDF files found: {len(pdf_files)}")
    for pdf in pdf_files:
        print(f"  - {pdf.name}")


✅ All imports successful!
📁 Working directory: /Users/youssefjanjar/Documents/formascience/class_parser
📂 Volume directory exists: True
📂 Slides directory exists: True
📄 PDF files found: 3
  - cours_4_plan.pdf
  - cours_1.pdf
  - cours_4.pdf


In [22]:
# Dev hot-reload setup (no kernel restart needed)
%load_ext autoreload
%autoreload 2

import sys, importlib
from pathlib import Path
# Ensure project root is on sys.path
sys.path.insert(0, str(Path('.').resolve()))

import src
import src.models as models
import src.llm as llm
import src.data_extraction as data_extraction
import src.course as course_mod
import src.pipeline as pipeline_mod


def refresh_imports():
    """Reload modules and rebind notebook aliases."""
    importlib.reload(src)
    importlib.reload(models)
    importlib.reload(llm)
    importlib.reload(data_extraction)
    importlib.reload(course_mod)
    importlib.reload(pipeline_mod)
    
    globals().update({
        # Models
        "Content": models.Content,
        "ContentSection": models.ContentSection,
        "MappingItem": models.MappingItem,
        "SectionSlideMapping": models.SectionSlideMapping,
        "Slides": models.Slides,
        "OutlineAndMapping": models.OutlineAndMapping,
        # LLM
        "OutlineOneShot": llm.OutlineOneShot,
        "OutlineTwoPass": llm.OutlineTwoPass,
        "MappingTwoPass": llm.MappingTwoPass,
        "Writer": llm.Writer,
        "PromptManager": llm.PromptManager,
        # Deterministic extractors
        "SlidesExtractor": data_extraction.SlidesExtractor,
        "PlanExtractor": data_extraction.PlanExtractor,
        # Orchestrators
        "Course": course_mod.Course,
        "CoursePipeline": pipeline_mod.CoursePipeline,
    })
    print("🔁 Imports refreshed")

# Initial bind
refresh_imports()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 Imports refreshed


## Step 1: Data Extraction - Slides

Extract slides from PDF using the exact logic from poc.ipynb


In [5]:
# Test slides extraction
pdf_path = "./volume/slides/cours_1.pdf"  # Update this path as needed

# Initialize extractor
slides_extractor = SlidesExtractor(
    min_avg_len=10,
    max_lines=20,
    merge_tol=2.0
)

print(f"🔍 Extracting slides from: {pdf_path}")

try:
    # Extract slides as Slides objects
    slides = slides_extractor.extract_slides(pdf_path)
    print(f"✅ Extracted {len(slides)} slides")
    
    # Display first few slides
    print("\n📋 First 3 slides:")
    for i, slide in enumerate(slides[:3]):
        print(f"\n--- Slide {i+1}: {slide.id} ---")
        print(f"Title: {slide.title}")
        print(f"Content (first 200 chars): {slide.content[:200]}...")
        print("=" * 50)
        
except FileNotFoundError:
    print(f"❌ PDF file not found: {pdf_path}")
    print("Please ensure the PDF file exists in ./volume/slides/")
    slides = []  # Empty list for testing
except Exception as e:
    print(f"❌ Error extracting slides: {e}")
    slides = []


🔍 Extracting slides from: ./volume/slides/cours_1.pdf
2025-08-17 16:02:26,950 [DEBUG] src.data_extraction.slides_extractor: Starting slide extraction from ./volume/slides/cours_1.pdf
2025-08-17 16:02:27,002 [DEBUG] src.data_extraction.slides_extractor: PDF opened, found 111 pages
2025-08-17 16:02:27,744 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P412' is an invalid float value
2025-08-17 16:02:27,746 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P414' is an invalid float value
2025-08-17 16:02:27,761 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P444' is an invalid float value
2025-08-17 16:02:27,822 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P451' is an invalid float value
2025-08-17 16:02:27,823 [WARNING] pdfminer.pdfinterp: Cannot set gray non-stroke color because /'P455' is an invalid float value
2025-08-17 16:02:27,838 [WARNING] pdfminer.pdfinterp: Cannot set gray non-

# Single Shot

# Single shot:  Extract the plan for this class 

In [6]:
outline_one_shot = OutlineOneShot()

outline, mapping = outline_one_shot.build_outline_and_mapping(slides[:20])

print(outline.print_outline())

2025-08-17 16:02:39,149 [DEBUG] src.llm.outline_one_shot: Starting one-shot outline and mapping generation with 20 slides
2025-08-17 16:04:34,809 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-17 16:04:34,867 [DEBUG] src.llm.outline_one_shot: Successfully generated outline with 5 sections and mapping with 13 entries
Course Outline:
Présentation du module et plan du cours
Introduction au contexte évolutif et organismes modèles
  Arbre de la vie et concept de LUCA
  Arbre des eucaryotes et choix d'organismes modèles
Notions fondamentales et définitions clés
  Homologie: définition, orthologie, paralogie et méthodes de détection
  Biologie du gène, dogme central et omiques
Le génome humain: composition et organisation générale
  Composition générale du génome: haploïde et diploïde
Détermination et séquençage du génome humain
  Historique, objectifs et ciblage de l’assemblage initial
  Cartographie génétique basée sur fréquences de recombinai

In [11]:
outline 

Content(sections=[ContentSection(id='SEC_1', title='Présentation du module et plan du cours', content=['L1SpS: UE 2 Les molécules du vivant\nU.E.2 Les molécules du vivant\n7. Organisation du génome humain, méthodes en\nbiotechnologie\n7.1 Architecture du génome\nhumain\nJean Muller\nLaboratoire Diagnostic Génétique (HUS)\nLaboratoire de Génétique médicale (Inserm U1112)\njeanmuller@unistra.fr', 'L1SpS: UE 2 Les molécules du vivant\nPlan du cours\n• Introduction\n• Notions fondamentales\n• Le génome humain\n• Projet de séquençage\n• Architecture globale\n• Génome mitochondrial\n• Génome nucléaire\n• Description des principaux éléments constituants\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\n• Comparaison aux autres génomes\n• Les types de variations du génome humain et leurs conséquences\n• La variabilité du génome humain\n• Evolution des génomes: notions essentielles, mécanismes\n2'], subsections=[]), ContentSection(id='SEC_2', title='Introduction au contexte év

In [12]:
print(mapping.visualize_mapping(outline))

Section-to-Slides Mapping:

[Root] SEC_1
Title: Présentation du module et plan du cours
Slides: 2 slide(s)
Slide IDs: SL_001, SL_002

[Root] SEC_2
Title: Introduction au contexte évolutif et organismes modèles
Slides: 1 slide(s)
Slide IDs: SL_003

  [Level 1] SEC_2.1
  Title: Arbre de la vie et concept de LUCA
  Slides: 1 slide(s)
  Slide IDs: SL_004

  [Level 1] SEC_2.2
  Title: Arbre des eucaryotes et choix d'organismes modèles
  Slides: 1 slide(s)
  Slide IDs: SL_005

[Root] SEC_3
Title: Notions fondamentales et définitions clés
Slides: 1 slide(s)
Slide IDs: SL_006

  [Level 1] SEC_3.1
  Title: Homologie: définition, orthologie, paralogie et méthodes de détection
  Slides: 5 slide(s)
  Slide IDs: SL_006, SL_007, SL_008, SL_009, SL_010

  [Level 1] SEC_3.2
  Title: Biologie du gène, dogme central et omiques
  Slides: 2 slide(s)
  Slide IDs: SL_011, SL_012

[Root] SEC_4
Title: Le génome humain: composition et organisation générale
Slides: 1 slide(s)
Slide IDs: SL_013

  [Level 1] SEC_

# Single shot: Writing content from mapping and the outline

In [13]:
outline_with_slides = outline.enrich_with_slides(slides=slides, mapping=mapping)
writer = Writer()

course = writer.write_course(enriched_content=outline_with_slides)

type(course)

2025-08-17 16:07:26,856 [DEBUG] src.llm.writer: Starting content enhancement for 5 sections
2025-08-17 16:10:02,399 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-17 16:10:02,491 [DEBUG] src.llm.writer: Successfully enhanced content with 5 sections


src.models.Content

In [24]:
type(course)

src.models.Content

In [25]:
print(course.print_content())

Course Content:

[Root] ID: SEC_1
Title: Présentation du module et plan du cours
Content:
  [1] Ce module porte sur l'organisation du génome humain et sur les méthodes
      biotechnologiques permettant de le caractériser et de l'analyser. Il vise
      à donner une vision intégrée de la composition, de l'architecture et de la
      dynamique des génomes, ainsi que des conséquences fonctionnelles et
      médicales des variations génomiques.
  [2] Le plan du cours présente successivement une introduction aux concepts
      évolutifs et aux organismes modèles, les notions fondamentales de
      génétique moléculaire, la description du génome humain (nucléaire et
      mitochondrial), le projet de séquençage, l'architecture globale du génome,
      la description des éléments constitutifs (gènes codants et non codants,
      pseudogènes, éléments répétés), la comparaison intergénomique, les
      différents types de variations et leurs conséquences, la variabilité du
      génome humain 

# Store a course as a artifact that I can load again into Course

In [35]:
refresh_imports()
# Get the raw data
content_data = course.model_dump()

# Recreate with current Content class
from src.models import Content
fresh_content = Content.model_validate(content_data)

pilot_course = Course(
    name="Architecture du génome humain",
    course_title="Architecture du génome humain",
    level="L1",
    block="SANTE",
    semester="S1",
    subject="UE-1 Constitution et transformation de la matière",
    chapter="CHAPITRE_9",
    content=fresh_content
)

pilot_course

🔁 Imports refreshed


Course(name='Architecture du génome humain', course_title='Architecture du génome humain', level='L1', block='SANTE', semester='S1', subject='UE-1 Constitution et transformation de la matière', chapter='CHAPITRE_9', content=Content(sections=[ContentSection(id='SEC_1', title='Présentation du module et plan du cours', content=["Ce module porte sur l'organisation du génome humain et sur les méthodes biotechnologiques permettant de le caractériser et de l'analyser. Il vise à donner une vision intégrée de la composition, de l'architecture et de la dynamique des génomes, ainsi que des conséquences fonctionnelles et médicales des variations génomiques.", "Le plan du cours présente successivement une introduction aux concepts évolutifs et aux organismes modèles, les notions fondamentales de génétique moléculaire, la description du génome humain (nucléaire et mitochondrial), le projet de séquençage, l'architecture globale du génome, la description des éléments constitutifs (gènes codants et non

In [36]:
pilot_course.save_to_json()

💾 Course saved to: volume/artifacts/architecture_du_génome_humain_20250817_162748.json


'volume/artifacts/architecture_du_génome_humain_20250817_162748.json'

In [32]:
# Check what's actually in your fresh_content variable
print(f"Type: {type(fresh_content)}")
print(f"Is Content? {isinstance(fresh_content, Content)}")

# Check the sections
if hasattr(fresh_content, 'sections'):
    print(f"Sections type: {type(fresh_content.sections)}")
    if fresh_content.sections:
        print(f"First section type: {type(fresh_content.sections[0])}")
        print(f"First section: {fresh_content.sections[0]}")
else:
    print("No sections attribute")

# Check if it's the right Content class
print(f"Content class: {Content}")
print(f"fresh_content's content class: {type(fresh_content).__module__}.{type(fresh_content).__name__}")

Type: <class 'src.models.Content'>
Is Content? True
Sections type: <class 'list'>
First section type: <class 'src.models.ContentSection'>
First section: id='SEC_1' title='Présentation du module et plan du cours' content=["Ce module porte sur l'organisation du génome humain et sur les méthodes biotechnologiques permettant de le caractériser et de l'analyser. Il vise à donner une vision intégrée de la composition, de l'architecture et de la dynamique des génomes, ainsi que des conséquences fonctionnelles et médicales des variations génomiques.", "Le plan du cours présente successivement une introduction aux concepts évolutifs et aux organismes modèles, les notions fondamentales de génétique moléculaire, la description du génome humain (nucléaire et mitochondrial), le projet de séquençage, l'architecture globale du génome, la description des éléments constitutifs (gènes codants et non codants, pseudogènes, éléments répétés), la comparaison intergénomique, les différents types de variation

# Two-shots

# Two-shots

## Plan from page

Extract plan text from PDF for Branch A (two-pass approach)


## Extract the slides

In [52]:
pdf_path = './volume/slides/cours_4.pdf'
plan_path = './volume/slides/cours_4_plan.pdf'

extractor = SlidesExtractor()

slides = extractor.extract_slides(pdf_path=pdf_path)

slides[:5]



2025-08-17 11:52:25,795 [DEBUG] src.data_extraction.slides_extractor: Starting slide extraction from ./volume/slides/cours_4.pdf
2025-08-17 11:52:25,854 [DEBUG] src.data_extraction.slides_extractor: PDF opened, found 144 pages
2025-08-17 11:52:28,232 [DEBUG] src.data_extraction.slides_extractor: Successfully extracted 135 slides


[Slides(id='SL_001', title="Biochimie du Gène et de l'Expression Génique", content="Biochimie du Gène et de l'Expression Génique\n(13 h cours – 11 chapitres)\nDrs. Pascal Dollé, Didier Devys, Philippe Kastner\n(Institut de Génétique et de Biologie Moléculaire et\nCellulaire [IGBMC], Parc d'innovation, Illkirch)\nDr. Jean Muller (Inserm U 1112, Génétique Médicale,\nCRBS, Faculté de Médecine)"),
 Slides(id='SL_002', title='1. Structure et propriétés des acides nucléiques', content="P.\nDollé\nD.\nDevys\nSommaire / Plan du cours (1)\n1. Structure et propriétés des acides nucléiques\n2. Réplication de l'ADN\n3. Réparation de l'ADN (anomalies et mécanismes de correction)\n4. Recombinaison de l’ADN (définition, généralités)\n5. Transcription de l'ADN (synthèse des ARN)\n6. Modifications (maturation) de l'ARN chez les eucaryotes\n7. Régulation de la transcription\n8. Code génétique\n9. Traduction (synthèse des protéines) P. Kastner\n10. Organisation et évolution des génomes eucaryotes animaux

In [ ]:
# Test plan extraction
plan_extractor = PlanExtractor()

plan_txt = plan_extractor.extract_plan_from_pdf(pdf_path=plan_path)

## Build the Outline

In [55]:
out_builder = OutlineTwoPass()

outline = out_builder.build_outline(plan_txt)

2025-08-17 11:56:28,875 [DEBUG] src.llm.outline_two_pass: Starting outline generation from plan text (7176 chars)
2025-08-17 11:57:07,334 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-17 11:57:07,452 [DEBUG] src.llm.outline_two_pass: Successfully generated outline with 4 sections


In [56]:
print(outline.print_outline())

Course Outline:
I. STRUCTURE ET PROPRIÉTÉS DES ACIDES NUCLÉIQUES
  1. Les nucléotides
    1.1. Les bases azotées
      1.1.1. Les bases pyrimidiques
      1.1.2. Les bases puriques
      1.1.3. Les bases modifiées dans l’ADN et l’ARN
      1.1.4. Autres dérivés : molécules d’intérêt biologique et médical
      1.1.5. Propriétés des bases azotées
      1.1.6. Transformation chimique des bases
    1.2. Les nucléosides
      1.2.1. Le pentose
      1.2.2. La liaison osidique
      1.2.3. Nomenclature
    1.3. Les nucléotides
      1.3.1. Nomenclature
      1.3.2. Nucléotides d’intérêt biologique
  2. Les acides nucléiques
    2.1. La liaison phosphodiester
    2.2. Structure spatiale des acides desoxyribonucléiques (ADN)
    2.3. Propriétés des ADN
      2.3.1. État ionique et solubilité
      2.3.2. Dénaturation de l’ADN / Effet sur l’absorbance
    2.4. Organisation des ADN
      2.4.1. Organisation de l’ADN viral
      2.4.2. Organisation de l’ADN des bactéries
      2.4.3. Organisatio

## Build the mapping

In [57]:
mapping_builder = MappingTwoPass()

mapping = mapping_builder.build_mapping(slides=slides, outline=outline) 

type(mapping)

2025-08-17 11:58:42,991 [DEBUG] src.llm.mapping_two_pass: Starting mapping generation for 135 slides and 4 sections
2025-08-17 12:05:15,276 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-17 12:05:15,317 [DEBUG] src.llm.mapping_two_pass: Successfully generated mapping with 83 entries


src.models.SectionSlideMapping

In [60]:
course_with_slides = outline.enrich_with_slides(slides=slides, mapping=mapping)


print(course_with_slides.print_outline())

Course Outline:
I. STRUCTURE ET PROPRIÉTÉS DES ACIDES NUCLÉIQUES
  1. Les nucléotides
    1.1. Les bases azotées
      1.1.1. Les bases pyrimidiques
      1.1.2. Les bases puriques
      1.1.3. Les bases modifiées dans l’ADN et l’ARN
      1.1.4. Autres dérivés : molécules d’intérêt biologique et médical
      1.1.5. Propriétés des bases azotées
      1.1.6. Transformation chimique des bases
    1.2. Les nucléosides
      1.2.1. Le pentose
      1.2.2. La liaison osidique
      1.2.3. Nomenclature
    1.3. Les nucléotides
      1.3.1. Nomenclature
      1.3.2. Nucléotides d’intérêt biologique
  2. Les acides nucléiques
    2.1. La liaison phosphodiester
    2.2. Structure spatiale des acides desoxyribonucléiques (ADN)
    2.3. Propriétés des ADN
      2.3.1. État ionique et solubilité
      2.3.2. Dénaturation de l’ADN / Effet sur l’absorbance
    2.4. Organisation des ADN
      2.4.1. Organisation de l’ADN viral
      2.4.2. Organisation de l’ADN des bactéries
      2.4.3. Organisatio

## Write the course

In [61]:
writer = Writer()

course = writer.write_course(enriched_content=course_with_slides)
type(course)

2025-08-17 12:09:44,106 [DEBUG] src.llm.writer: Starting content enhancement for 4 sections
2025-08-17 12:14:38,352 [INFO] httpx: HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-08-17 12:14:38,538 [DEBUG] src.llm.writer: Successfully enhanced content with 4 sections


src.models.Content

In [62]:
print(course.print_content())

Course Content:

[Root] ID: SEC_1
Title: I. STRUCTURE ET PROPRIÉTÉS DES ACIDES NUCLÉIQUES
Content:
  [1] Les acides nucléiques sont des polymères de nucléotides qui constituent le
      support moléculaire de l'information génétique et les effecteurs de son
      expression. Historiquement, la « nucléine » a été identifiée au XIXe
      siècle et la nature chimique de l'acide désoxyribonucléique (ADN) a été
      précisée ultérieurement ; la distinction fondamentale entre ADN et ARN
      repose sur la nature du pentose et la composition en bases azotées.
  [2] Sur le plan fonctionnel, l'ADN est le support de l'information génétique
      chez la plupart des organismes tandis que l'ARN joue, selon les contextes
      biologiques, des rôles d'information (ARN messagers), de traduction (ARN
      de transfert), de structure et catalyse (ARN ribosomiques et autres ARN
      non codants). Le chapitre 1 introduit ces notions de base et s'inscrit
      dans un cursus couvrant la réplication,

In [1]:
course

NameError: name 'course' is not defined

## Step 3: Branch B - One-Shot Approach (No Plan)

Test the one-shot outline and mapping generation when no plan is provided


In [15]:
# Test Branch B: One-shot approach
if slides:
    print("🚀 Testing Branch B: One-shot outline and mapping generation")
    
    # Initialize one-shot generator
    one_shot = OutlineOneShot(model="gpt-5-mini")
    
    try:
        print("🤖 Generating outline and mapping in one shot...")
        outline_b, mapping_b = one_shot.build_outline_and_mapping(slides)  # Use first 10 slides for testing
        
        print(f"✅ One-shot generation successful!")
        print(f"📋 Outline sections: {len(outline_b.sections)}")
        print(f"🗺️ Mapping entries: {len(mapping_b.mapping)}")
        
        # Display outline structure
        print("\n📋 Generated Outline:")
        print(outline_b.print_outline())
        
        # Display mapping
        print("\n🗺️ Generated Mapping:")
        for item in mapping_b.mapping[:20]:  # Show first 5 mappings
            print(f"  {item.section_id} → {item.slide_ids}")
        
        # Store for later use
        branch_b_outline = outline_b
        branch_b_mapping = mapping_b
        
    except Exception as e:
        print(f"❌ Error in one-shot generation: {e}")
        branch_b_outline = None
        branch_b_mapping = None

        
else:
    print("⚠️ Skipping Branch B test - no slides available")
    branch_b_outline = None
    branch_b_mapping = None


🚀 Testing Branch B: One-shot outline and mapping generation
🤖 Generating outline and mapping in one shot...
outline=Content(sections=[ContentSection(id='SEC_1', title='Introduction et objectifs du cours sur le génome humain', content=[], subsections=[ContentSection(id='SEC_1.1', title='Présentation et organisation du cours', content=[], subsections=[]), ContentSection(id='SEC_1.2', title='Introduction au thème et contexte biologique', content=[], subsections=[])]), ContentSection(id='SEC_2', title='Notions fondamentales en évolution et homologie génétique', content=[], subsections=[ContentSection(id='SEC_2.1', title='Arbre de la vie, évolution et sélection naturelle', content=[], subsections=[]), ContentSection(id='SEC_2.2', title='Homologie, orthologie et paralogie: définitions et méthodes', content=[], subsections=[]), ContentSection(id='SEC_2.3', title='Dogme central et référentiels moléculaires', content=[], subsections=[])]), ContentSection(id='SEC_3', title='Le génome humain: str

In [16]:
# Display mapping
print("\n🗺️ Generated Mapping:")
for item in mapping_b.mapping[:20]:  # Show first 5 mappings
    print(f"  {item.section_id} → {item.slide_ids}")


🗺️ Generated Mapping:
  SEC_1 → ['SL_001']
  SEC_1.1 → ['SL_002']
  SEC_1.2 → ['SL_003']
  SEC_2 → ['SL_004']
  SEC_2.1 → ['SL_004', 'SL_005']
  SEC_2.2 → ['SL_006', 'SL_007', 'SL_008', 'SL_009', 'SL_010']
  SEC_2.3 → ['SL_011', 'SL_012']
  SEC_3 → ['SL_013']
  SEC_3.1 → ['SL_014', 'SL_015', 'SL_016', 'SL_017']
  SEC_3.2 → ['SL_018', 'SL_019', 'SL_020', 'SL_021', 'SL_022', 'SL_024', 'SL_025', 'SL_026', 'SL_027', 'SL_028', 'SL_029']
  SEC_3.3 → ['SL_030', 'SL_031', 'SL_032', 'SL_033', 'SL_034', 'SL_035']
  SEC_4 → ['SL_036']
  SEC_4.1 → ['SL_037', 'SL_038', 'SL_039', 'SL_040', 'SL_041', 'SL_042', 'SL_043', 'SL_044', 'SL_045', 'SL_046', 'SL_047']
  SEC_4.2 → ['SL_048']
  SEC_4.3 → ['SL_049', 'SL_050', 'SL_051', 'SL_052', 'SL_053']
  SEC_4.4 → ['SL_054', 'SL_055', 'SL_056', 'SL_057', 'SL_058', 'SL_059', 'SL_060', 'SL_061', 'SL_062', 'SL_063', 'SL_064', 'SL_065', 'SL_066']
  SEC_5 → ['SL_068']
  SEC_5.1 → ['SL_067', 'SL_069', 'SL_070', 'SL_071', 'SL_072']
  SEC_5.2 → ['SL_073']
  SEC_6 → 

## Step 3.5: Content Enrichment (using Content.enrich_with_slides)

Note: No separate ContentEnricher class is needed; enrichment is a method on Content.


In [17]:
# Demonstrate enrichment method directly on Content
# Use whatever outline/mapping you have (prefer Branch B if available)

if 'branch_b_outline' in locals() and branch_b_outline and 'branch_b_mapping' in locals() and branch_b_mapping and slides:
    enriched_demo = branch_b_outline.enrich_with_slides(slides, branch_b_mapping)
    print("✅ Enrichment via Content.enrich_with_slides complete (Branch B)")
    print(f"Sections enriched: {len(enriched_demo.sections)}")
elif 'branch_a_outline' in locals() and branch_a_outline and 'branch_a_mapping' in locals() and branch_a_mapping and slides:
    enriched_demo = branch_a_outline.enrich_with_slides(slides, branch_a_mapping)
    print("✅ Enrichment via Content.enrich_with_slides complete (Branch A)")
    print(f"Sections enriched: {len(enriched_demo.sections)}")
else:
    print("⚠️ Skipping enrichment demo - no outline/mapping available")
    enriched_demo = None


✅ Enrichment via Content.enrich_with_slides complete (Branch B)
Sections enriched: 9


In [ ]:
type(enriched_demo)

writer = Writer()
written_course = writer.write_course(enriched_demo)

print(written_course.print_content())

## Step 4: Complete Pipeline Test

Test the full CoursePipeline end-to-end for both branches


In [26]:
print(plan_text)

   L1SpS: UE 2 Les molécules du vivant                                                             
      Plan      du     cours                                                                       
        •  Introduction                                                                            
             •  Notions  fondamentales                                                             
        •  Le génome    humain                                                                     
             •  Projet de séquençage                                                               
             •  Architecture globale                                                               
                  •  Génome    mitochondrial                                                       
                  •  Génome    nucléaire                                                           
             •  Description  des principaux  éléments   constituants                               


In [27]:
two_pass = OutlineTwoPass()

outline = two_pass.build_outline(plan_text)

print(outline.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
  Description des principaux éléments constituants
    Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


In [ ]:
mapping_two_pass = MappingTwoPass()

mapping = mapping_two_pass.build_mapping(slides, outline)


In [ ]:
print(mapping.visualize_mapping(outline))

In [ ]:
course_with_slides = outline.enrich_with_slides(slides, mapping)
print(course_with_slides.print_content())

In [ ]:
writer = Writer()

final_course = writer.write_course(course_with_slides)

print(final_course.print_content())

In [36]:
print(final_course.print_content())

Course Content:

[Root] ID: SEC_1
Title: Introduction
Content:
  [1] Ce module propose une introduction générale aux molécules du vivant en
      prenant le génome humain comme point d'appui pour aborder notions
      fondamentales, architecture moléculaire, variabilité et mécanismes
      évolutifs. Le plan couvre successivement les concepts de base, la
      description du génome, l'histoire du séquençage, l'organisation nucléaire
      et mitochondriale, les différents types d'éléments composant le génome,
      les variations et leur interprétation, ainsi que les processus d'évolution
      génomique.
  [2] Le génome constitue l'une des références biologiques majeures d'un
      organisme et s'articule avec d'autres couches d'information fonctionnelle
      : le transcriptome (ensemble des ARN exprimés), le méthylome (marquages
      épigénétiques) et le protéome (ensemble des protéines). L'étude conjointe
      de ces niveaux permet de comprendre la régulation des gènes, l'express

## Step 5: Summary and Production Example

Summary of all tests and example of how to use the system in production
